In [1]:
%matplotlib widget
from tqdm.auto import tqdm, trange
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import math
from numpy.random import default_rng, SeedSequence
import multiprocessing
import concurrent.futures
from numba import vectorize,jit,prange,objmode,njit

In [2]:
rng = default_rng()

In [3]:
error = 1e-10 #because machine error is *really* important when working with linalg

In [4]:
def normalize(v):
    l = np.linalg.norm(v)
    if l < error:
        print("Something has gone terribly wrong with "+str(v))
        return v
    return v/l

Questions for next version:
* Make precise:
    * Why probability 1? Give intuition and what measure 0 set works
    * Say what $x_n, p_n, v_n$ are, give picture
    * Conjecture 4.1: Provide evidence and convince the reader that, for irrational starting direction 
        * Get some random [convex polygons](https://cglab.ca/~sander/misc/ConvexGeneration/convex.html) and see if that works
    * Check conj 4.2 for irregular triangles
* Density of paths:
    * How to measure?
        * Maybe try making circles and finding the largest radius $\epsilon$ s.t. $B_\epsilon \cap L =\emptyset$ (for $L$ is the set of points in the lines)
    * How dense are random paths on 7-gons? Show with 1000 bounces
    * Are there protected regions in small $n$ $n$-gons? If so, how?

### Defining shapes
In general we will define a discrete shape to be an array of pairs of points which represent the lines between them.

In [5]:
def polar_poly(r,n):
    r = np.vectorize(r)
    theta = 2*np.pi/n
    angles = np.array([x*theta for x in range(n)])
    p0s = np.vstack((r(angles)*np.cos(angles),r(angles)*np.sin(angles))).T
    p1s = np.array(list(map(lambda i: p0s[(i+1)%n],range(n))))
    return np.array(list(zip(p0s,p1s)))

In [6]:
fig = plt.figure()
shape = polar_poly(lambda t: 1+np.cos(t),6)
plt.plot(*shape.T)
plt.gca().set_aspect(1)
plt.plot(0,0,'ro')
plt.axis('off')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
def ngon(n):
    return polar_poly(lambda t: 1,n)

In [8]:
fig = plt.figure()
shape = ngon(5)
plt.plot(*shape.T)
plt.gca().set_aspect(1)
plt.plot(0,0,'ro')
plt.axis('off')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
def circle_arcs(n):
    f = lambda t: np.array([np.cos(t),np.sin(t)])
    fs = []
    ns = []
    tbounds = []
    for i in range(n):
        fs.append(f)
        ns.append(f)
        tbounds.append((2*np.pi/i,2*np.pi/(i+1)))
    return (fs,ns,tbound)

In [10]:
def polar_curve(r,rprime):
    rhat = lambda t: np.array([np.cos(t),np.sin(t)])
    that = lambda t: rhat(t+np.pi/2)
    f = lambda t: r(t)*np.array([np.cos(t),np.sin(t)])
    n = lambda t: -r(t)*rhat(t)+rprime(t)*that(t)
    tbound = (0,2*np.pi)
    return ([f],[n],[tbound])

General Hit Algorithms

In [11]:
#@njit(fastmath=True,parallel=True)
def line_hit(x,v,p0,p1):
    dp = p0-p1#np.subtract(p0,p1)
    a = np.vstack((v,dp)).T
    b = p0-x#np.subtract(p0,x)
    t = np.linalg.solve(a,b)
    nx = x+t[0]*v #np.add(x,np.multiply(t[1],v))
    p = p0-t[1]*dp#np.add(p0,np.multiply(t[0],dp))
    if t[1] < 1 and t[1] > 0 and t[0] > 0 and np.linalg.norm(p-nx) < error:
        n = np.array([-dp[1],dp[0]])
        return True, (p,n,(x,v,t[0]))
    else:
        return False, ()

In [12]:
#@jit(fastmath=True,parallel=True,forceobj=True)
def curve_hit(x,v,f,n,tbound):
    g = lambda st: (x+v*st[0])-f(st[1])
    hit,_,_,i,m = fsolve[g,[1,0.5*tbound[0]+0.5*tbound[1]]]
    if i == 1:
        r = normalize(n(hit[1]))
        p = x+v*hit[0]
        return True, (p,r,(x,v,hit[1]))
    return False, ()

Filtering specific hits of piecewise curves/shapes

In [13]:
#@jit(fastmath=True,parallel=True,forceobj=True)
def general_hit(hits):
    crossings = list(map(lambda x: (x[1],x[2]), filter(lambda x: x[0], hits)))
    if crossings == []:
        raise Exception("No hits!")
    cross = sorted(crossings,key=lambda x: x[0][2][2])[0]
    p = cross[0][0]
    n = cross[0][1]
    s = cross[1]
    t = cross[0][2][2]
    return p, n, s, t

In [14]:
def polygon_hit(ps):
    def hit(x,v):
        hits = list(map(lambda p: line_hit(x,v,*p[1])+(p[0],),enumerate(ps))) #append i to the side
        return general_hit(hits)
    return hit

In [15]:
def piecewise_smooth(fs,ns,tbounds):
    def hit(x,v):
        hits = list(map(lambda ifnt: curve_hit(x,v,ifnt[1],ifnt[2],ifnt[3])+(ifnt[0],),enumerate(zip(fs,ns,tbounds))))
        return general_hit(hits)
    return hit

Special Case

In [16]:
#@njit(fastmath=True,parallel=True)
def circle_hit(x,v):
    b = np.dot(v,x)
    c = np.linalg.norm(x)**2 -1
    ts = -b+np.array([1,-1])*np.sqrt(b**2 - c)
    t = np.max(ts)#np.min(ts[np.all(np.greater(ts,[0,0]))])
    p = x+t*v
    n = -p
    return p, n, np.arctan2(x[1],x[0])

Random Convex Shapes

In [17]:
def rand_convex(n):
    x = np.sort(rng.random(size=n)*2-1)
    y = np.sort(rng.random(size=n)*2-1)
    minx,maxx = x[0],x[-1]
    miny,maxy = y[0],y[-1]
    
    xbs = rng.choice([True,False],size=n-2)
    x0 = x[1:-1]
    x1 = x0[xbs]
    x2 = x0[np.logical_not(xbs)]
    
    ybs = rng.choice([True,False],size=n-2)
    y0 = y[1:-1]
    y1 = y0[ybs]
    y2 = y0[np.logical_not(ybs)]
    
    x1 = np.hstack((minx,x1,maxx))
    x2 = np.hstack((minx,x2,maxx))
    xs = np.array([])
    for i,x in enumerate(x1[:-1]):
        xs = np.append(xs,x1[i+1]-x)
    for i,x in enumerate(x2[:-1]):
        xs = np.append(xs,x-x2[i+1])
        
    y1 = np.hstack((miny,y1,maxy))
    y2 = np.hstack((miny,y2,maxy))
    ys = np.array([])
    for i,y in enumerate(y1[:-1]):
        ys = np.append(ys,y1[i+1]-y)
    for i,y in enumerate(y2[:-1]):
        ys = np.append(ys,y-y2[i+1])
        
    rng.shuffle(ys)
    
    vec = list(zip(xs,ys))
    vec.sort(key=lambda x: np.arctan2(x[1],x[0]))
    vec = np.array(vec)
    
    p = [0,0]
    p0s = np.array([p])
    for v in vec:
        p = p+v
        p0s = np.append(p0s,[p],axis=0)
        
    del p
    off = np.array([minx,miny]) - np.min(p0s,axis=0)
    p0s = np.array(list(map(lambda p: p+off,p0s)))
    p1s = np.array(list(map(lambda i: p0s[(i+1)%n],range(n))))
    
    return np.array(list(zip(p0s,p1s)))

In [18]:
fig = plt.figure()
shape = rand_convex(5)
plt.plot(*shape.T)
plt.gca().set_aspect(1)
plt.plot(0,0,'ro')
plt.axis('off')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Bounce algorithm

In [19]:
#@njit(fastmath=True,parallel=True)
def bounce(v,n):
    n = normalize(n)
    return v-2*np.dot(n,v)*n#np.subtract(v,np.multiply(2*np.dot(n,v),n))

Simulation of many bounces

In [20]:
#@jit(fastmath=True,parallel=True,forceobj=True)
def sim(hitFunc,x,v,depth):
    lines = [(x,v,-1)]
    for _ in range(depth):
        x, n, s, t = hitFunc(x,v)
        v = bounce(v,n)
        x = x+error*v#np.add(x,np.multiply(1e-12,v))
        lines = lines + [(x,v,s)]
    return lines

In [21]:
def dynamical_sim(hitFunc,x,v,side,depth):
    lines = [x]
    hits = [0]
    for _ in range(depth):
        x, n, s, t = hitFunc(x,v)
        v = bounce(v,n)
        x = x+error*v
        if s == side:
            hits = hits + [t]
        lines = lines + [x]
    return lines,hits[1:]

Random Start Info

In [22]:
def randx(rad):
    r = rng.random()*rad
    theta = 2*np.pi*rng.random()
    x = np.multiply(r,np.array([np.cos(theta),np.sin(theta)]))
    return x
def randv():
    vtheta = 2*np.pi*rng.random()
    v = np.array([np.cos(vtheta),np.sin(vtheta)])
    return v

Get data for plotting

In [23]:
def side_chance(sim_result,mod=0):
    if mod == 0:
        s = np.array(list(map(lambda x: x[2],sim_result[1:])))
    else:
        s = np.array(list(map(lambda x: x[2]%mod,sim_result[1:])))
    uniques = np.unique(s)
    counts = np.array(list(map(lambda u: sum(1 for side in s if side == u),uniques)))
    counts = normalize(counts)
    return uniques,counts

Dynamical Analysis

In [24]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
shape = ngon(4)#10*rand_convex(5)
shapehit = polygon_hit(shape)
ts = np.linspace(0,np.pi/8)
for t in tqdm(ts):    
    ls,hs = dynamical_sim(shapehit,np.array([0.25-1,0.24]),np.array([np.cos(t),np.sin(t)]),0,10000)
    xns = hs[1:]
    xn1s = hs[:-1]
    ax.scatter(xns,xn1s,t,marker=matplotlib.markers.MarkerStyle(marker=','))
    xy = np.linspace(0,max(hs))
    #ax.plot(xy,xy,t,'r--')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Random N-Gon Sims

In [25]:
def sl(q):
    return np.linalg.norm(q[1]-q[0])
sidelen = np.vectorize(sl,signature='(2,2)->()')
#@njit(fastmath=True,parallel=True)
def lenprob(depth,sides,trials,tries=50):
    length = np.array([])
    prob = np.array([])
    theta = np.array([])
    
    for n in sides:
        for k in trange(trials):
            shape = 10*rand_convex(n+3)
            hit = polygon_hit(shape)
            sidelengths = sidelen(shape)
            t = 0
            ls = None
            while t < tries:
                t += 1
                v = randv()
                x = randx(1)
                try:
                    ls = sim(hit,x,v,depth)
                except:
                    continue
                t = tries+1
                us,cs = side_chance(ls)
                ncs = cs/np.sum(sidelengths)
                length = np.append(length,sidelengths[us])
                prob = np.append(prob,ncs)
                theta = np.append(theta,np.arctan2(v[1],v[0]))
        print("Got to {} sides".format(n))
    return length, prob, theta

In [27]:
plt.figure()
l,p,_ = lenprob(1000,[100],1000,tries=1)
fit = np.polyfit(l,p,1)
poly = np.poly1d(fit)
lenth = np.linspace(0,max(l))
print("{}x+{}".format(fit[0],fit[1]))
plt.plot(l,p,'k,',label="100-gons")
plt.plot(lenth,poly(lenth),'r--',label="Best Fit")
plt.legend()
plt.savefig('lenprob100.png')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …


Got to 100 sides
0.0019447743039847241x+2.9197308882922995e-05


In [ ]:
fig = plt.figure()
nums = [3,4,5,6,7,8,9,10]
colors = "krbgcmy"
for n,c in zip(nums,colors):
    l,p,_ = lenprob(10000,[n],5000)
    plt.plot(l,p,c+',',label="{}-gons".format(n))
    fit = np.polyfit(l,p,1)
    print("{}x+{}".format(fit[0],fit[1]))
plt.legend()
plt.savefig('lenprob.png')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …


Got to 3 sides
0.0008228402645866947x+0.0016015890311828077



Got to 4 sides
0.0008436980550192983x+0.001125140162847277


In [ ]:
#from mpl_toolkits.mplot3d import Axes3D
#fig = plt.figure()
#ax = fig.add_subplot(111, projection='3d')
#ax.scatter(l,p,a)
#plt.savefig('lenprobangle.png')
#plt.show()

Specific Square Sims

In [ ]:
sq = ngon(4)
sqhit = polygon_hit(sq)
x0 = np.array([0.25-1,0.24])
slopes = [np.sqrt(2)/2,np.sqrt(3),np.sqrt(5)]
n = 100
for m in slopes:
    ls = sim(sqhit,x0,np.array([m,1]),n)
    plt.plot(*sq.T)
    xs = list(map(lambda x: x[0][0], ls))
    ys = list(map(lambda x: x[0][1], ls))
    plt.plot(xs,ys)
    x,v,_ = ls[-1]
    plt.quiver(*x,*v)
    ax = plt.gca()
    ax.set_aspect(1)
    plt.axis('off')
    plt.savefig('{}for{}.png'.format(np.round(m,3),n))
    plt.show()

In [ ]:
ss = list(map(lambda x: x[2]%2,ls[1:]))
#print(ss)
#ax = plt.gca()
#ax.hist(ss,bins=2*sides)
us,cs = side_chance(ls,mod=2)
plt.bar(us,cs)

Specifically Circular Sims

In [ ]:
deg = 30
theta = np.pi/2-(np.pi/180)*deg #<- Edit this!
ls = sim(circle_hit,np.array([0,1]),np.array([np.cos(theta),np.sin(theta)]),1000)
xs = list(map(lambda x: x[0][0], ls))
ys = list(map(lambda x: x[0][1], ls))
#print(xs)
plt.plot(xs,ys)
print(ls[-1])
x,v,_ = ls[-1]
plt.quiver(*x,*v)
draw_circle = plt.Circle((0, 0), 1,fill=False)
plt.xlim(-1,1)
plt.ylim(-1,1)
plt.gcf().gca().add_artist(draw_circle)
plt.gca().set_aspect(1)
plt.axis('off')
plt.savefig("{}deg.png".format(deg))
plt.show()

In [ ]:
ss = list(map(lambda x: x[2],ls[1:]))
#print(ss) #<- Side sequence
#ax = plt.gca()
#ax.hist(ss,bins=bounces)
us,cs = side_chance(ls)
plt.bar(us,cs,width=0.05)

Discrete Cardiod Sims

In [ ]:
shape = polar_poly(lambda t: 1+np.cos(t),100)
shapehit = polygon_hit(shape)
ls = sim(shapehit,randx(0.5)+np.array([0.75,0]),randv(),10000)
#ls = sim(shapehit,np.array([0.,0.]),np.array([0.,1.]),100)
plt.plot(*shape.T)
xs = list(map(lambda x: x[0][0], ls))
ys = list(map(lambda x: x[0][1], ls))
plt.plot(xs,ys)
x,v,_ = ls[-1]
plt.quiver(*x,*v)
ax = plt.gca()
ax.set_aspect(1)
plt.axis('off')
plt.savefig('6cardiodrand.png')
plt.show()

In [ ]:
ss = list(map(lambda x: x[2],ls[1:]))
#print(ss)
#ax = plt.gca()
#ax.hist(ss,bins=bounces)
sidelengths = np.array(list(map(lambda q: np.linalg.norm(q[1]-q[0]),shape)))
print(sidelengths)
us,cs = side_chance(ls)
print(sum(cs))
ncs = cs/sum(cs)#/sidelengths[us]
plt.bar(us,ncs,width=0.75)
plt.plot(us,sidelengths[us]/sum(sidelengths),'r')
plt.savefig("6normcardiodsidesplot.png")

In [ ]:
#(fs,ns,ts) = polar_curve(lambda t: 1+np.cos(t),lambda t: -np.sin(t))
#hit = piecewise_smooth(fs,ns,ts)
#ls = sim(hit,randx(0.5)+np.array([0.75,0]),randv(),1000)
ts = np.linspace(0,2*np.pi)
xs = (1+np.cos(ts))*np.cos(ts)
ys = (1+np.cos(ts))*np.sin(ts)
plt.plot(xs,ys)
#xs = list(map(lambda x: x[0][0], ls))
#ys = list(map(lambda x: x[0][1], ls))
#plt.plot(xs,ys)
#x,v,_ = ls[-1]
#plt.quiver(*x,*v)
ax = plt.gca()
ax.set_aspect(1)

Setting up shapes etc.

In [ ]:
sides = 110
bounces = 1000
ng = ngon(sides)
nghit = polygon_hit(ng)

Random Circle & N-Gon

In [ ]:
ls = sim(nghit,randx(np.cos(np.pi/sides)),randv(),bounces)
plt.plot(*ng.T)
xs = list(map(lambda x: x[0][0], ls))
ys = list(map(lambda x: x[0][1], ls))
plt.plot(xs,ys)
x,v,_ = ls[-1]
plt.quiver(*x,*v)
plt.gca().set_aspect(1)
plt.axis('off')
plt.savefig('ngonirrational2.png')
plt.show()

In [ ]:
ss = list(map(lambda x: x[2],ls[1:]))
#print(ss)
#ax = plt.gca()
#ax.hist(ss,bins=2*sides)
us,cs = side_chance(ls)
plt.bar(us,cs)
plt.savefig('100.png')

In [ ]:
ls = sim(circle_hit,randx(0.9),randv(),bounces)
xs = list(map(lambda x: x[0][0], ls))
ys = list(map(lambda x: x[0][1], ls))
plt.plot(xs,ys)
print(ls[-1])
x,v,_ = ls[-1]
plt.quiver(*x,*v)
draw_circle = plt.Circle((0, 0), 1,fill=False)
plt.gcf().gca().add_artist(draw_circle)
plt.gca().set_aspect(1)
plt.axis('off')
plt.show()

In [ ]:
ss = list(map(lambda x: x[2],ls[1:]))
#print(ss)
#ax = plt.gca()
#ax.hist(ss,bins=bounces)
us,cs = side_chance(ls)
plt.bar(us,cs,width=0.05)